In [1]:
# Function to calculate the inside temperature for the next hour given current inside, outside temperature and power that the termostat is set to
# param T_in: Inside temperature based on last hour
# param T_out: Outside temperature based on forecast
# param k: Based on historical electricity use in business we solve for the constant
# param c_cool: We calculate a constant as conservation of total energy does not apply in this situation in this system
# param cm: Approximation for mass of air multiplied by heat capacity
# param V: Approximated volume of office building

# Power is now in kilowattitunti
# 


function T_c(T_in, T_out, power)
    Q_in = power # kWH
    k= 8/22 # [k] = kWh/K
    c_cool = 0.7264 #Dimensionless constant
    Q_out = k *(T_out - T_in) #The unit is kWh/K
    V = 12000 # m^3
    rho = 1.204 # kg/m^3
    c = 700/(3*10^6)
    cm = rho*c*V
    return (Q_in + Q_out)/cm*2.6 + T_in*c_cool

end

function P_bad(T_bad, T_out)
    tehohävio = (T_bad-T_out)*0.25*9.6
    return(10+tehohävio/2)
end

P_bad (generic function with 1 method)

In [2]:
using JuMP, Cbc
# Function to optimize heating of business building during a 24h period
# param o: Array of length 24, 1 if someone present else 0
# param p: Preference for saving energy over money
# param Ep: Hourly next day prize of electricity
# param T_opt: Optimal temperature
# param Pt: Acceptable deviation from optimal temperature
# param T_bad: Setting of the temperature for non-optimal model (assumed to be constant)
# param T_out_vec: Vector of forcasted outside temperatures for the comming day
# param T_end: Previous day last hour temperature
# param T_n_low: Lowest acceptable night temperature
# param T_n_hi: Highest acceptable night temperature
# param C: Constant denoting the proportion of people present and total working force at office

function Twenty_four(o, p, Ep, T_opt, Pt, T_bad, T_out_vec, T_end, T_n_low, T_n_hi, C,s1,s2)
    
    # Initialize vector for saving the hourly development of the temperature
    #Tc = Vector{Float64}(undef,25)
    #Tc[1] = T_end
   
    Tc_arr = zeros(24)
    Tc_arr[1] = T_end
    P_bad_arr = zeros(24)
    Optimal_vec = zeros(24)
    # Chained multi-parameter LP optimization
    for i in 1:24 
        Tc = Tc_arr[i]
        hourly_optimum = Model(with_optimizer(Cbc.Optimizer, logLevel = 0)) # Create the model for optimizing the hourly optimum thermostat setting
        
        @variable(hourly_optimum, power >= 0)

        P_bad_new = P_bad(T_bad,T_out_vec[i])
        Tc_new = T_c(Tc,T_out_vec[i],power)
        # Define constraints for money
        #@constraint(hourly_optimum,P_bad_new*Ep[i]-power*Ep[i] >= 0)
        
        # Define constraints for energy
        @constraint(hourly_optimum,Tc_new - (T_opt-(1-o[i])*(T_opt-T_n_low)) >= 0)
        @constraint(hourly_optimum,Tc_new - (T_opt+(1-o[i])*(T_n_hi-T_opt)) <= 0)
        
        #Objective function for optimization problem. We maximize a multiparameter problem for each hour, with terms energy saved, money saved and 
        @objective(hourly_optimum, Max, (1-p)*s1*(P_bad_new*Ep[i]-power*Ep[i]) + p*s2*(P_bad_new-power) - (T_opt - Tc_new)*C[i])
        
        optimize!(hourly_optimum);
        Optimal_vec[i] = value.(power)
        Tc_arr[i] = value.(Tc_new)
        P_bad_arr[i] = value.(P_bad_new)
    end
    return (Optimal_vec,Tc_arr,P_bad_arr)
end

Twenty_four (generic function with 1 method)

In [3]:
using CSV
using DataFrames 
using Statistics

# Read the data
C_df = DataFrame(CSV.File("/home/hannuls3/notebooks/Juction-2022-OTTER/Data/people_constant_factor.csv")) 
o_df = DataFrame(CSV.File("/notebooks/Juction-2022-OTTER/Data/people_flow_binary.csv")) 
E_df = DataFrame(CSV.File("/home/hannuls3/notebooks/Juction-2022-OTTER/Whole_year_electricity.csv",header =0)) 
o_df=o_df[:,2:365]
C_df=C_df[:,2:365]
T_out_vec = DataFrame(CSV.File("/home/hannuls3/notebooks/Juction-2022-OTTER/Data/penis.csv",header = 0))


,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,9.05,10.01,10.01,10.02,11.09,11.02,12.05,11.04,11.05
2,9.03,10.02,10.01,10.01,11.09,11.03,12.04,11.04,11.05
3,9.03,10.05,10.01,9.08,11.07,11.0,11.09,11.04,11.05
4,9.03,10.04,9.09,9.06,11.08,10.08,11.04,11.05,11.05
5,8.07,10.02,9.03,9.03,11.07,10.07,11.01,11.05,11.05
6,8.05,9.09,9.04,9.03,11.06,11.0,11.0,11.05,11.04
7,8.08,10.0,10.0,9.09,11.06,10.08,10.07,11.06,11.05
8,8.08,10.06,10.07,10.03,11.04,10.09,11.05,11.07,11.08
9,9.04,11.03,11.02,10.08,10.08,11.06,11.05,11.06,11.09


In [10]:
# For loop for extending the prediction to the whole year

preds = zeros(24,365)
temps = zeros(24,365)
base = zeros(24,365)

for i in 1:364
    tout = 19
    println(i)
    (preds[:,i], temps[:,i], base[:,i])= Twenty_four(o_df[:,i],0.5, E_df[:,i], 21.0, 2.0,21.0, T_out_vec[:,i], tout, 17.2, 25.0, C_df[:,i],100/0.0053,1000/0.29)
    tout = temps[24,i]
end



#CSV.write("/home/hannuls3/notebooks/Juction-2022-OTTER/Data/Optimized-data/ptimal_thermos.csv", DataFrame(preds, :auto))
#CSV.write("/home/hannuls3/notebooks/Juction-2022-OTTER/Data/Optimized-data/temp_data.csv", DataFrame(temps, :auto))
#CSV.write("/home/hannuls3/notebooks/Juction-2022-OTTER/Data/Optimized-data/base_data.csv", DataFrame(base, :auto))

1
Presolve 0 (-2) rows, 0 (-1) columns and 0 (-2) elements
Optimal - objective value -11055987
After Postsolve, objective -11055987, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -11055987.3 - 0 iterations time 0.002, Presolve 0.00
Presolve 0 (-2) rows, 0 (-1) columns and 0 (-2) elements
Optimal - objective value -25997072
After Postsolve, objective -25997072, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -25997071.78 - 0 iterations time 0.002, Presolve 0.00
Presolve 0 (-2) rows, 0 (-1) columns and 0 (-2) elements
Optimal - objective value -25764396
After Postsolve, objective -25764396, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -25764396.08 - 0 iterations time 0.002, Presolve 0.00
Presolve 0 (-2) rows, 0 (-1) columns and 0 (-2) elements
Optimal - objective value -17579723
After Postsolve, objective -17579723, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -17579722.92 - 0 iterations time 0.002, Presolve 0.00
Presolve 0 (-2)

Excessive output truncated after 526437 bytes.

Presolve 0 (-2) rows, 0 (-1) columns and 0 (-2) elements
Optimal - objective value -62256782
After Postsolve, objective -62256782, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -62256782.2 - 0 iterations time 0.002, Presolve 0.00
Presolve 0 (-2) rows, 0 (-1) columns and 0 (-2) elements
Optimal - objective value -47484170
After Postsolve, objective -47484170, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -47484169.61 - 0 iterations time 0.002, Presolve 0.00
Presolve 0 (-2) rows, 0 (-1) columns and 0 (-2) elements
Optimal - objective value -64680036
After Postsolve, objective -64680036, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -64680036.21 - 0 iterations time 0.002, Presolve 0.00
Presolve 0 (-2) rows, 0 (-1) columns and 0 (-2) elements
Optimal - objective value -62627703
After Postsolve, objective -62627703, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -62627703.27 - 0 iterations time 0.002, Presolve 0.00
Presolve 0 (-2) r

"/home/hannuls3/notebooks/Juction-2022-OTTER/Data/Optimized-data/base_data.csv"